In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, RobertaTokenizer, RobertaForSequenceClassification
import torch

# RoBERATa and GPT-2 to Analysis Sentiment and Text Polishing

In [71]:
def load_roberta_model():
    tokenizer = RobertaTokenizer.from_pretrained('roberta_saved_model')
    model = RobertaForSequenceClassification.from_pretrained('roberta_saved_model', num_labels=4)# 4 labels: work, friend, family, sentiment
    model.eval() 
    return tokenizer, model

# Sentiment and Scenario Prediction Function
def predict_sentiment_and_scenario(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.sigmoid(logits) #Multi-label classification, use sigmoid activation function

    #  Predicted labels: work, friend, family, sentiment
    work, friend, family, sentiment_numeric = predictions[0].tolist()

    # Sentiment prediction: positive or negative
    sentiment = "positive" if sentiment_numeric >= 0.5 else "negative"

    # Scenario prediction: work, friend, family, other
    if work >= 0.5:
        scenario = "work"
    elif friend >= 0.5:
        scenario = "friend"
    elif family >= 0.5:
        scenario = "family"
    else:
        scenario = "other"
    
    return sentiment, scenario

# Load RoBERTa model
tokenizer, roberta_model = load_roberta_model()

# Input text 
text_input = "I don’t think you’re right for the job because you’re doing it all wrong."
predicted_sentiment, predicted_scenario = predict_sentiment_and_scenario(roberta_model, tokenizer, text_input)

print(f"Predicted Sentiment: {predicted_sentiment}, Predicted Scenario: {predicted_scenario}")

Predicted Sentiment: negative, Predicted Scenario: work


Code from https://huggingface.co/docs/transformers/model_doc/gpt2

In [73]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 model and tokenizer
def load_gpt2_model():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    return tokenizer, model

# Refine negative text based on sentiment and scenario
def refine_negative_text(model, tokenizer, scenario, original_text):
    if scenario == "work":
        prompt = f"Refine this message in a polite and professional way: {original_text}"
    elif scenario == "friend":
        prompt = f"Refine this message to sound more friendly and casual: {original_text}"
    elif scenario == "family":
        prompt = f"Refine this message to sound warm and understanding: {original_text}"
    
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=70, num_return_sequences=1)
    refined_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return refined_text

# Load GPT-2 model and tokenizer
gpt2_tokenizer, gpt2_model = load_gpt2_model()

# If the sentiment is negative, refine the text, otherwise, no refinement needed
if predicted_sentiment == "negative":
    refined_text = refine_negative_text(gpt2_model, gpt2_tokenizer, predicted_scenario, text_input)
    print(f"Refined Text:\n{refined_text}")
else:
    print("The input text is already positive, no refinement needed.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Refined Text:
Refine this message in a polite and professional way: I don’t think you’re right for the job because you’re doing it all wrong. I’t think you’re doing it all wrong because you’re doing it all wrong because you’re doing it all wrong because you’


In [28]:
!pip install rouge-score bert-score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for bert-score from https://files.pythonhosted.org/packages/c6/8c/bc5457de4c004b1a623b31f7bc8d0375fb699b7d67df11879098b4b7b7c8/bert_score-0.3.13-py3-none-any.whl.metadata
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
   --------------------------------- ------ 51.2/61.1 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 61.1/61.1 kB 1.1 MB/s eta 0:00:00
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24970 sha256=8b27ebb202b4445fb80d2d22de4d10cd45426e45480351550187e4dd08cb0f92
  Stored in directory: c:\users\tobys\appdata\local\pi

In [10]:
# Original text and refined text
original_text = text_input
if predicted_sentiment == "negative":
    generated_text = refined_text
else:
    generated_text = text_input # If the text is positive, no refinement needed



In [12]:
from rouge_score import rouge_scorer
from bert_score import score

In [74]:
original_text = "I don’t think you’re right for the job because you’re doing it all wrong."  # Original text
reference_text = "I think you are doing it all wrong." # Reference text

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(original_text, reference_text)

print(f"ROUGE-1 F1: {rouge_scores['rouge1'].fmeasure:.4f}")
print(f"ROUGE-2 F1: {rouge_scores['rouge2'].fmeasure:.4f}")
print(f"ROUGE-L F1: {rouge_scores['rougeL'].fmeasure:.4f}")

# Calculate BERTScore
P, R, F1 = score([original_text], [reference_text], lang="en", verbose=True)

print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall: {R.mean().item():.4f}")
print(f"BERTScore F1: {F1.mean().item():.4f}")

ROUGE-1 F1: 0.5600
ROUGE-2 F1: 0.3478
ROUGE-L F1: 0.5600


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.66 seconds, 0.38 sentences/sec
BERTScore Precision: 0.9159
BERTScore Recall: 0.9645
BERTScore F1: 0.9396


## Code Reference List

Hugging Face (n.d.). OpenAI GPT2. [online] huggingface.co. Available at: https://huggingface.co/docs/transformers/model_doc/gpt2 [Accessed 19 Sep. 2024].